#### 说明：
$\bullet\quad$本章主要涉及的Python文件是：$\text{debugger_defines.py}$，$\text{debugger.py}$和$\text{test.py}$  

$\bullet\quad$$\text{debugger_defines.py}$存放调试器所需的结构体、联合体和常量，直接使用源代码包中的文件

$\bullet\quad$$\text{debugger.py}$实现我们的Windows下的轻量级调试器，逐步完善

$\bullet\quad$$\text{test.py}$是调试器逐步完善过程中的测试脚本(测试套件)，逐步完善

## 1.调试器直接调用程序

In [ ]:
# debugger.py
# Run the executable from the debugger itself

from ctypes import *
from debugger_defines import *

kernel32 = windll.kernel32

class debugger():

    def __init__(self):
        pass

    def load(self, path_to_exe):
        
        # dwCreation flag determines how to create the process
        # set creation_flags = CREATE_NEW_CONSOLE if you want
        # to see the calculator GUI
        creation_flags = DEBUG_PROCESS
        
        # instantiate the structs
        startupinfo         = STARTUPINFO()
        process_information = PROCESS_INFORMATION()
        
        # The following two options allow the started process
        # to be shown as a separate window. This also illustrates
        # how different settings in the STARTUPINFO struct can affect
        # the debuggee.
        startupinfo.dwFlags     = 0x1
        startupinfo.wShowWindow = 0x0
        
        # We then initialize the cb variable in the STARTUPINFO struct
        # which is just the size of the struct itself
        startupinfo.cb = sizeof(startupinfo)
        
        if kernel32.CreateProcessA(path_to_exe,
                                    None,
                                    None,
                                    None,
                                    None,
                                    creation_flags,
                                    None,
                                    None,
                                    byref(startupinfo),
                                    byref(process_information)):
            print "[*] we have successfully launched the process!"
            print "[*] PID:%d" % process_information.dwProcessId
        else:
            print "[*] Error:0x%08x." % kernel32.GetLastError()

In [ ]:
# test.py
# run the executable from the debugger itself

import debugger

debugger = debugger.debugger()

debugger.load(r"C:\Windows\System32\calc.exe")

#### 说明：
$\bullet\quad$由于$\text{debugger_defines.py}$使用的是源代码包中的文件，较长，故暂时不展示出来

$\bullet\quad$$\text{debugger_defines.py}$中定义的结构体、联合体和常量，将有另外的文档进行说明

$\bullet\quad$$\text{debugger.py}$中调用的$\text{Microsoft Win32 API}$，将有另外的文档进行说明

#### 运行测试：

If you execute this Python file either via the command line or from your IDE, it will spawn the process you entered, report the process identifier (PID), and then exit. If you use my example of calc.exe, you will not see the calculator’s GUI appear. The reason you won’t see the GUI is because the process hasn’t painted it to the screen yet, because it is waiting for the debugger to continue execution. We haven’t built the logic to do that yet, but it’s coming soon! You now know how to spawn a process that is ready to be debugged. It’s time to whip up some code that attaches a debugger to a running process.

## 2.调试器附加到指定进程

In [ ]:
# debugger.py
# Attach the debugger to the specified process
# Such as calc.exe

from ctypes import *
from debugger_defines import *

kernel32 = windll.kernel32

class debugger():

    def __init__(self):
        pass


    def load(self, path_to_exe):

        # dwCreation flag determines how to create the process
        # set creation_flags = CREATE_NEW_CONSOLE if you want
        # to see the calculator GUI
        creation_flags = DEBUG_PROCESS

        # instantiate the structs
        startupinfo         = STARTUPINFO()
        process_information = PROCESS_INFORMATION()

        # The following two options allow the started process
        # to be shown as a separate window. This also illustrates
        # how different settings in the STARTUPINFO struct can affect
        # the debuggee.
        startupinfo.dwFlags     = 0x1
        startupinfo.wShowWindow = 0x0

        # We then initialize the cb variable in the STARTUPINFO struct
        # which is just the size of the struct itself
        startupinfo.cb = sizeof(startupinfo)

        if kernel32.CreateProcessA(path_to_exe,
                                    None,
                                    None,
                                    None,
                                    None,
                                    creation_flags,
                                    None,
                                    None,
                                    byref(startupinfo),
                                    byref(process_information)):
            print "[*] we have successfully launched the process!"
            print "[*] PID:%d" % process_information.dwProcessId
        else:
            print "[*] Error:0x%08x." % kernel32.GetLastError()

            
            
# --------------------------- Start New Codes ---------------------------
    def open_process(self, pid):

        h_process = kernel32.OpenProcess(PROCESS_ALL_ACCESS, pid, False)
        return h_process


    def attach(self, pid):

        self.h_process = self.open_process(pid)

        # We attempt to attach to the process
        # if this fails we exit the call
        if kernel32.DebugActiveProcess(pid):
            self.debugger_active = True
            self.pid             = int(pid)
            self.run()
        else:
            print "[*] Unable to attach to the process."


    def run(self):

        # Now we have to poll the debuggee for
        # debugging events
        while self.debugger_active == True:
            self.get_debug_event()


    def get_debug_event(self):

        debug_event     = DEBUG_EVENT()
        continue_status = DBG_CONTINUE

        if kernel32.WaitForDebugEvent(byref(debug_event), INFINITE):
            # We aren't going to build any event handlers
            # just yet. Let's just resume the process for now.
            raw_input("Press a key to continue...")
            self.debugger_active = False
            kernel32.ContinueDebugEvent(debug_event.dwProcessId,debug_event.dwThreadId, continue_status)


    def detach(self):

        if kernel32.DebugActiveProcessStop(self.pid):
            print "[*] Finished debugging. Exiting..."
        else:
            print "There was an error!"
            return False
# ---------------------------  End  New Codes ---------------------------

#### 说明：

$\bullet\quad$新增的类函数：$\text{open_process}$、$\text{attach}$、$\text{run}$、$\text{get_debug_event}$、$\text{detach}$

In [ ]:
# test.py
# Attach the debugger to the specified process
# Such as calc.exe

import debugger

debugger = debugger.debugger()

#debugger.load(r"C:\Windows\System32\calc.exe")                          # Comment out

# --------------------------- Start New Codes ---------------------------
pid = raw_input("Enter the PID of the process to attach to: ")

debugger.attach(int(pid))

debugger.detach()
# ---------------------------  End  New Codes ---------------------------

#### 说明：

$\bullet\quad$注释掉行$\text{debugger.load(r"C:\Windows\System32\calc.exe")}$

$\bullet\quad$新增最后三行

#### 运行测试：

$\bullet\quad$运行VMware中的Windows XP系统的计算器程序

$\bullet\quad$运行任务管理，查看计算器进程(calc.exe)的PID

$\bullet\quad$执行$\text{test.py}$脚本，出现$\text{Enter the PID of the process to attach to:}$时输入上述PID

$\bullet\quad$当出现$\text{Press a key to continue...}$的提示时，尝试操作计算器，没有任何响应

$\bullet\quad$在控制台中输入任意键，屏幕打印$\text{[*] Finished debugging. Exiting...}$， 然后$\text{test.py}$执行结束

$\bullet\quad$现在能正常操作计算器

如果上述程序正常运行，则注释掉$\text{debugger.py}$中：

$\quad\bullet\quad$类函数$\text{attach}$

In [ ]:
def attach(self, pid):

    self.h_process = self.open_process(pid)

    # We attempt to attach to the process
    # if this fails we exit the call
    if kernel32.DebugActiveProcess(pid):
        self.debugger_active = True
        self.pid             = int(pid)
#        self.run()                      # Comment out
    else:
        print "[*] Unable to attach to the process."

$\quad\bullet\quad$类函数$\text{get_debug_event}$

In [ ]:
def get_debug_event(self):

    debug_event     = DEBUG_EVENT()
    continue_status = DBG_CONTINUE

    if kernel32.WaitForDebugEvent(byref(debug_event), INFINITE):
        # We aren't going to build any event handlers
        # just yet. Let's just resume the process for now.
#        raw_input("Press a key to continue...")      # Comment out
#        self.debugger_active = False                 # Comment out
        
        kernel32.ContinueDebugEvent(debug_event.dwProcessId,
                                    debug_event.dwThreadId, 
                                    continue_status)

## 3.获得CPU寄存器状态

In [ ]:
# debugger.py
# Obtaining CPU Register State

from ctypes import *
from debugger_defines import *

kernel32 = windll.kernel32

class debugger():

    def __init__(self):
        pass


    def load(self, path_to_exe):

        # dwCreation flag determines how to create the process
        # set creation_flags = CREATE_NEW_CONSOLE if you want
        # to see the calculator GUI
        creation_flags = DEBUG_PROCESS

        # instantiate the structs
        startupinfo         = STARTUPINFO()
        process_information = PROCESS_INFORMATION()

        # The following two options allow the started process
        # to be shown as a separate window. This also illustrates
        # how different settings in the STARTUPINFO struct can affect
        # the debuggee.
        startupinfo.dwFlags     = 0x1
        startupinfo.wShowWindow = 0x0

        # We then initialize the cb variable in the STARTUPINFO struct
        # which is just the size of the struct itself
        startupinfo.cb = sizeof(startupinfo)

        if kernel32.CreateProcessA(path_to_exe,
                                    None,
                                    None,
                                    None,
                                    None,
                                    creation_flags,
                                    None,
                                    None,
                                    byref(startupinfo),
                                    byref(process_information)):
            print "[*] we have successfully launched the process!"
            print "[*] PID:%d" % process_information.dwProcessId
        else:
            print "[*] Error:0x%08x." % kernel32.GetLastError()


    def open_process(self, pid):

        h_process = kernel32.OpenProcess(PROCESS_ALL_ACCESS, pid, False)
        return h_process


    def attach(self, pid):

        self.h_process = self.open_process(pid)

        # We attempt to attach to the process
        # if this fails we exit the call
        if kernel32.DebugActiveProcess(pid):
            self.debugger_active = True
            self.pid             = int(pid)
#            self.run()                    # Comment out
        else:
            print "[*] Unable to attach to the process."


    def run(self):

        # Now we have to poll the debuggee for
        # debugging events
        while self.debugger_active == True:
            self.get_debug_event()


    def get_debug_event(self):

        debug_event     = DEBUG_EVENT()
        continue_status = DBG_CONTINUE

        if kernel32.WaitForDebugEvent(byref(debug_event), INFINITE):
            # We aren't going to build any event handlers
            # just yet. Let's just resume the process for now.
#            raw_input("Press a key to continue...")         # Comment out
#            self.debugger_active = False                    # Comment out

            kernel32.ContinueDebugEvent(debug_event.dwProcessId,
                                        debug_event.dwThreadId,
                                        continue_status)


    def detach(self):

        if kernel32.DebugActiveProcessStop(self.pid):
            print "[*] Finished debugging. Exiting..."
        else:
            print "There was an error!"
            return False

        
# --------------------------- Start New Codes ---------------------------
    def open_thread (self, thread_id):

        h_thread = kernel32.OpenThread(THREAD_ALL_ACCESS, None, thread_id)

        if h_thread is not None:
            return h_thread
        else:
            print "[*] Could not obtain a valid thread handle."
            return False


    def enumerate_threads(self):

        thread_entry     = THREADENTRY32()
        thread_list      = []
        snapshot         = kernel32.CreateToolhelp32Snapshot(TH32CS_SNAPTHREAD, self.pid)

        if snapshot is not None:

            # You have to set the size of the struct
            # or the call will fail
            thread_entry.dwSize = sizeof(thread_entry)

            success = kernel32.Thread32First(snapshot, byref(thread_entry))

            while success:
                if thread_entry.th32OwnerProcessID == self.pid:
                    thread_list.append(thread_entry.th32ThreadID)

                success = kernel32.Thread32Next(snapshot, byref(thread_entry))

            # No need to explain this call, it closes handles
            # so that we don't leak them.
            kernel32.CloseHandle(snapshot)
            return thread_list
        else:
            return False


    def get_thread_context (self, thread_id=None,h_thread=None):

        context = CONTEXT()
        context.ContextFlags = CONTEXT_FULL | CONTEXT_DEBUG_REGISTERS

        # Obtain a handle to the thread
        if h_thread is None:
            self.h_thread = self.open_thread(thread_id)

        if kernel32.GetThreadContext(self.h_thread, byref(context)):

            return context
        else:
            return False
# ---------------------------  End  New Codes ---------------------------

#### 说明：

$\bullet\quad$新增的类函数：$\text{open_thread}$、$\text{enumerate_threads}$、$\text{get_thread_context}$

In [ ]:
# test.py
# Obtaining CPU Register State

import debugger

debugger = debugger.debugger()

#debugger.load(r"C:\Windows\System32\calc.exe")

pid = raw_input("Enter the PID of the process to attach to: ")

debugger.attach(int(pid))

# --------------------------- Start New Codes ---------------------------
list = debugger.enumerate_threads()

# For each thread in the list we want to
# grab the value of each of the registers
for thread in list:
    thread_context = debugger.get_thread_context(thread)

    # Now let's output the contents of some of the registers
    print "[*] Dumping registers for thread ID: 0x%08x" % thread
    print "[**] EIP: 0x%08x" % thread_context.Eip
    print "[**] ESP: 0x%08x" % thread_context.Esp
    print "[**] EBP: 0x%08x" % thread_context.Ebp
    print "[**] EAX: 0x%08x" % thread_context.Eax
    print "[**] EBX: 0x%08x" % thread_context.Ebx
    print "[**] ECX: 0x%08x" % thread_context.Ecx
    print "[**] EDX: 0x%08x" % thread_context.Edx
    print "[*] END DUMP"
# ---------------------------  End  New Codes ---------------------------   
    
debugger.detach()

#### 运行测试：

## 4.1实现调试事件处理--获得事件代码值与线程ID

In [ ]:
# debugger.py
# Implementing Debug Event Handlers -get the event code and thread(s) ID
# Show which event has been fired based on the event code.
# Using this information, we will be able to see the general flow
# of events after we have spawned or attached to a process.

from ctypes import *
from debugger_defines import *

kernel32 = windll.kernel32

class debugger():

    def __init__(self):
#         pass                                # Comment Out
# --------------------------- Start New Codes ---------------------------
        self.h_process       = None
        self.pid             = None
        self.debugger_active = False
        self.h_thread        = None
        self.context         = None
# ---------------------------  End  New Codes ---------------------------   


    def load(self, path_to_exe):

        # dwCreation flag determines how to create the process
        # set creation_flags = CREATE_NEW_CONSOLE if you want
        # to see the calculator GUI
        creation_flags = DEBUG_PROCESS

        # instantiate the structs
        startupinfo         = STARTUPINFO()
        process_information = PROCESS_INFORMATION()

        # The following two options allow the started process
        # to be shown as a separate window. This also illustrates
        # how different settings in the STARTUPINFO struct can affect
        # the debuggee.
        startupinfo.dwFlags     = 0x1
        startupinfo.wShowWindow = 0x0

        # We then initialize the cb variable in the STARTUPINFO struct
        # which is just the size of the struct itself
        startupinfo.cb = sizeof(startupinfo)

        if kernel32.CreateProcessA(path_to_exe,
                                    None,
                                    None,
                                    None,
                                    None,
                                    creation_flags,
                                    None,
                                    None,
                                    byref(startupinfo),
                                    byref(process_information)):
            print "[*] we have successfully launched the process!"
            print "[*] PID:%d" % process_information.dwProcessId
        else:
            print "[*] Error:0x%08x." % kernel32.GetLastError()


    def open_process(self, pid):

        h_process = kernel32.OpenProcess(PROCESS_ALL_ACCESS, pid, False)
        return h_process


    def attach(self, pid):

        self.h_process = self.open_process(pid)

        # We attempt to attach to the process
        # if this fails we exit the call
        if kernel32.DebugActiveProcess(pid):
            self.debugger_active = True
            self.pid             = int(pid)
#            self.run()                    # Comment out
        else:
            print "[*] Unable to attach to the process."


    def run(self):

        # Now we have to poll the debuggee for
        # debugging events
        while self.debugger_active == True:
            self.get_debug_event()


    def get_debug_event(self):

        debug_event     = DEBUG_EVENT()
        continue_status = DBG_CONTINUE

        if kernel32.WaitForDebugEvent(byref(debug_event), INFINITE):
            # We aren't going to build any event handlers
            # just yet. Let's just resume the process for now.
#            raw_input("Press a key to continue...")         # Comment out
#            self.debugger_active = False                    # Comment out

# --------------------------- Start New Codes ----------------------------
            # Let's obtain the thread and context information
            self.h_thread          = self.open_thread(debug_event.dwThreadId)
            self.context           = self.get_thread_context(h_thread=self.h_thread)
            
            print "Event Code: %d Thread ID: %d" % \
                (debug_event.dwDebugEventCode,debug_event.dwThreadId)
# ---------------------------  End  New Codes ----------------------------

            kernel32.ContinueDebugEvent(debug_event.dwProcessId,
                                        debug_event.dwThreadId,
                                        continue_status)


    def detach(self):

        if kernel32.DebugActiveProcessStop(self.pid):
            print "[*] Finished debugging. Exiting..."
        else:
            print "There was an error!"
            return False


    def open_thread (self, thread_id):

        h_thread = kernel32.OpenThread(THREAD_ALL_ACCESS, None, thread_id)

        if h_thread is not None:
            return h_thread
        else:
            print "[*] Could not obtain a valid thread handle."
            return False


    def enumerate_threads(self):

        thread_entry     = THREADENTRY32()
        thread_list      = []
        snapshot         = kernel32.CreateToolhelp32Snapshot(TH32CS_SNAPTHREAD, self.pid)

        if snapshot is not None:

            # You have to set the size of the struct
            # or the call will fail
            thread_entry.dwSize = sizeof(thread_entry)

            success = kernel32.Thread32First(snapshot, byref(thread_entry))

            while success:
                if thread_entry.th32OwnerProcessID == self.pid:
                    thread_list.append(thread_entry.th32ThreadID)

                success = kernel32.Thread32Next(snapshot, byref(thread_entry))

            # No need to explain this call, it closes handles
            # so that we don't leak them.
            kernel32.CloseHandle(snapshot)
            return thread_list
        else:
            return False


    def get_thread_context (self, thread_id=None,h_thread=None):

        context = CONTEXT()
        context.ContextFlags = CONTEXT_FULL | CONTEXT_DEBUG_REGISTERS

        # Obtain a handle to the thread
        if h_thread is None:
            self.h_thread = self.open_thread(thread_id)

        if kernel32.GetThreadContext(self.h_thread, byref(context)):

            return context
        else:
            return False


In [ ]:
# test.py
# Implementing Debug Event Handlers -get the event code and thread(s) ID
# Show which event has been fired based on the event code.
# Using this information, we will be able to see the general flow
# of events after we have spawned or attached to a process.

import debugger

debugger = debugger.debugger()

#debugger.load(r"C:\Windows\System32\calc.exe")

pid = raw_input("Enter the PID of the process to attach to: ")

debugger.attach(int(pid))

#list = debugger.enumerate_threads()                                  # Comment out

## For each thread in the list we want to                             # Comment out
## grab the value of each of the registers                            # Comment out
#for thread in list:                                                  # Comment out
#    thread_context = debugger.get_thread_context(thread)             # Comment out
#                                                                     # Comment out
#    # Now let's output the contents of some of the registers         # Comment out
#    print "[*] Dumping registers for thread ID: 0x%08x" % thread     # Comment out
#    print "[**] EIP: 0x%08x" % thread_context.Eip                    # Comment out
#    print "[**] ESP: 0x%08x" % thread_context.Esp                    # Comment out
#    print "[**] EBP: 0x%08x" % thread_context.Ebp                    # Comment out
#    print "[**] EAX: 0x%08x" % thread_context.Eax                    # Comment out
#    print "[**] EBX: 0x%08x" % thread_context.Ebx                    # Comment out
#    print "[**] ECX: 0x%08x" % thread_context.Ecx                    # Comment out
#    print "[**] EDX: 0x%08x" % thread_context.Edx                    # Comment out
#    print "[*] END DUMP"                                             # Comment out

# --------------------------- Start New Codes ----------------------------
debugger.run()
# ---------------------------  End  New Codes ----------------------------

debugger.detach()

#### 运行测试：

#### 说明：

$\quad\bullet\quad$调试事件代码表

$\quad\bullet\quad$执行上述$\text{test.py}$脚本后控制台输出信息说明：

基于上述脚本输出:  
我们看到$\text{CREATE_PROCESS_DEBUG_EVENT(0x3)}$事件第一个发生，  
接下来是多次$\text{LOAD_DLL_DEBUG_EVENT(0x6)}$事件，  
然后是创建一个新线程的$\text{CREATE_THREAD_DEBUG_EVENT(x02)}$事件。

接着是$\text{EXCEPTION_DEBUG_EVENT}(x01)$异常事件，由Windows设置的断点所引发，允许在进程启动前观察进程的状态  
最后是一个结束线程事件是$\text{EXIT_THREAD_DEBUG_EVENT}(x04)$，由线程1340结束自身产生

异常事件非常重要，异常可能是断点，访问异常，或者是内存访问错误（例如尝试写到一个只读内存区）  
所有这些都很重要，让我们先捕捉第一个Windows设置的断点

## 4.2 实现调试事件处理--捕捉Windows设置的断点

In [ ]:
# debugger.py
# Catching the first Windows-driven breakpoint.

from ctypes import *
from debugger_defines import *

kernel32 = windll.kernel32

class debugger():

    def __init__(self):
#        pass
        self.h_process         = None
        self.pid               = None
        self.debugger_active   = False
        self.h_thread          = None
        self.context           = None
# --------------------------- Start New Codes ----------------------------
        self.exception         = None
        self.exception_address = None
# ---------------------------  End  New Codes ----------------------------


    def load(self, path_to_exe):

        # dwCreation flag determines how to create the process
        # set creation_flags = CREATE_NEW_CONSOLE if you want
        # to see the calculator GUI
        creation_flags = DEBUG_PROCESS

        # instantiate the structs
        startupinfo         = STARTUPINFO()
        process_information = PROCESS_INFORMATION()

        # The following two options allow the started process
        # to be shown as a separate window. This also illustrates
        # how different settings in the STARTUPINFO struct can affect
        # the debuggee.
        startupinfo.dwFlags     = 0x1
        startupinfo.wShowWindow = 0x0

        # We then initialize the cb variable in the STARTUPINFO struct
        # which is just the size of the struct itself
        startupinfo.cb = sizeof(startupinfo)

        if kernel32.CreateProcessA(path_to_exe,
                                    None,
                                    None,
                                    None,
                                    None,
                                    creation_flags,
                                    None,
                                    None,
                                    byref(startupinfo),
                                    byref(process_information)):
            print "[*] we have successfully launched the process!"
            print "[*] PID:%d" % process_information.dwProcessId
        else:
            print "[*] Error:0x%08x." % kernel32.GetLastError()


    def open_process(self, pid):

        h_process = kernel32.OpenProcess(PROCESS_ALL_ACCESS, pid, False)
        return h_process


    def attach(self, pid):

        self.h_process = self.open_process(pid)

        # We attempt to attach to the process
        # if this fails we exit the call
        if kernel32.DebugActiveProcess(pid):
            self.debugger_active = True
            self.pid             = int(pid)
#            self.run()                    # Comment out
        else:
            print "[*] Unable to attach to the process."


    def run(self):

        # Now we have to poll the debuggee for
        # debugging events
        while self.debugger_active == True:
            self.get_debug_event()


    def get_debug_event(self):

        debug_event     = DEBUG_EVENT()
        continue_status = DBG_CONTINUE

        if kernel32.WaitForDebugEvent(byref(debug_event), INFINITE):
            # We aren't going to build any event handlers
            # just yet. Let's just resume the process for now.
#            raw_input("Press a key to continue...")         # Comment out
#            self.debugger_active = False                    # Comment out

            # Let's obtain the thread and context information
            self.h_thread          = self.open_thread(debug_event.dwThreadId)
            self.context           = self.get_thread_context(h_thread=self.h_thread)

            print "Event Code: %d Thread ID: %d" % \
                (debug_event.dwDebugEventCode,debug_event.dwThreadId)
                
# --------------------------- Start New Codes ----------------------------
            if debug_event.dwDebugEventCode == EXCEPTION_DEBUG_EVENT:
                self.exception = debug_event.u.Exception.ExceptionRecord.ExceptionCode
                self.exception_address = debug_event.u.Exception.ExceptionRecord.ExceptionAddress

                # call the internal handler for the exception event that just occured.
                if self.exception == EXCEPTION_ACCESS_VIOLATION:
                    print "Access Violation Detected."
                # If a breakpoint is Detected,
                # we call an internal handler
                elif self.exception == EXCEPTION_BREAKPOINT:
                    continue_status = self.exception_handler_breakpoint()
                elif self.exception == EXCEPTION_GUARD_PAGE:
                    print "Guard Page Access Detected."
                elif self.exception == EXCEPTION_SINGLE_STEP:
                    print "Single Stepping."
# ---------------------------  End  New Codes ----------------------------

            kernel32.ContinueDebugEvent(debug_event.dwProcessId,
                                        debug_event.dwThreadId,
                                        continue_status)


    def detach(self):

        if kernel32.DebugActiveProcessStop(self.pid):
            print "[*] Finished debugging. Exiting..."
        else:
            print "There was an error!"
            return False


    def open_thread (self, thread_id):

        h_thread = kernel32.OpenThread(THREAD_ALL_ACCESS, None, thread_id)

        if h_thread is not None:
            return h_thread
        else:
            print "[*] Could not obtain a valid thread handle."
            return False


    def enumerate_threads(self):

        thread_entry     = THREADENTRY32()
        thread_list      = []
        snapshot         = kernel32.CreateToolhelp32Snapshot(TH32CS_SNAPTHREAD, self.pid)

        if snapshot is not None:

            # You have to set the size of the struct
            # or the call will fail
            thread_entry.dwSize = sizeof(thread_entry)

            success = kernel32.Thread32First(snapshot, byref(thread_entry))

            while success:
                if thread_entry.th32OwnerProcessID == self.pid:
                    thread_list.append(thread_entry.th32ThreadID)

                success = kernel32.Thread32Next(snapshot, byref(thread_entry))

            # No need to explain this call, it closes handles
            # so that we don't leak them.
            kernel32.CloseHandle(snapshot)
            return thread_list
        else:
            return False


    def get_thread_context (self, thread_id=None,h_thread=None):

        context = CONTEXT()
        context.ContextFlags = CONTEXT_FULL | CONTEXT_DEBUG_REGISTERS

        # Obtain a handle to the thread
        if h_thread is None:
            self.h_thread = self.open_thread(thread_id)

        if kernel32.GetThreadContext(self.h_thread, byref(context)):

            return context
        else:
            return False
        
        
# --------------------------- Start New Codes ----------------------------
    def exception_handler_breakpoint(self):

        print "[*] Hit the Windows-driven breakpoint."
        print "Exception Address: 0x%08x" % self.exception_address
        return DBG_CONTINUE
# ---------------------------  End  New Codes ----------------------------

#### 说明：

新增类函数：$\text{exception_handler_breakpoint}$

In [ ]:
# test.py
# Catching the first Windows-driven breakpoint.

import debugger

debugger = debugger.debugger()

pid = raw_input("Enter the PID of the process to attach to: ")

debugger.attach(int(pid))

debugger.run()

debugger.detach()

#### 说明：

上述$\text{test.py}$与4.1中的$\text{test.py}$完全一样，仅仅是删除了所有注释语句

#### 运行测试：

从上述控制台输出中，我们看到比4.1中的输出结果多了两行：

说明类函数$\text{get_debug_event}$捕捉到了软件断点事件并调用类函数$\text{exception_handler_breakpoint}$进行处理并输出上述信息